<a href="https://colab.research.google.com/github/Untick/Usedesk/blob/Yashin-Mikhail-branch/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2_AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка библиотек

In [2]:
import pandas as pd
from IPython.display import display
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Загрузка и подготовка данных

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_excel('/content/drive/MyDrive/StudyFiles/Витрина_Юздеск_3 (1).xlsx')
df.head()

,11803,comp_id_new,месяц_desc,дата,рейтинг,всего,есть ответ,выполнен,закрыт,nps,...,u-task,u_problem,u_incident,u_low,u_medium,u_urgent,u_extreme,licens,бз,бз_ст
0,1,1,1,2023-08-01,2,887,851,394,342,326,...,0,0,0,0,0,0,0,13,1,4
1,2,1,2,2023-07-01,2,5324,5138,146,4478,1923,...,0,0,0,0,4,0,0,13,1,4
2,3,1,3,2023-06-01,2,5376,5033,4,4557,1865,...,0,0,0,0,2,0,0,13,1,4
3,4,1,4,2023-05-01,2,5573,5301,1,4775,2007,...,0,0,0,0,5,0,0,13,1,4
4,5,1,5,2023-04-01,2,6138,5983,2,5500,2241,...,0,0,0,0,1,0,0,13,1,4


In [5]:
df = df.drop(['comp_id_new', 'месяц_desc','дата'], axis = 1)
df.head()

,11803,рейтинг,всего,есть ответ,выполнен,закрыт,nps,email,telephony,api,...,u-task,u_problem,u_incident,u_low,u_medium,u_urgent,u_extreme,licens,бз,бз_ст
0,1,2,887,851,394,342,326,290,0,0,...,0,0,0,0,0,0,0,13,1,4
1,2,2,5324,5138,146,4478,1923,2011,0,0,...,0,0,0,0,4,0,0,13,1,4
2,3,2,5376,5033,4,4557,1865,1949,0,0,...,0,0,0,0,2,0,0,13,1,4
3,4,2,5573,5301,1,4775,2007,2008,0,0,...,0,0,0,0,5,0,0,13,1,4
4,5,2,6138,5983,2,5500,2241,1991,0,7,...,0,0,0,0,1,0,0,13,1,4


In [6]:
df = df.drop(df.columns[0], axis = 1)
df.head()

,рейтинг,всего,есть ответ,выполнен,закрыт,nps,email,telephony,api,usedesk_chat,...,u-task,u_problem,u_incident,u_low,u_medium,u_urgent,u_extreme,licens,бз,бз_ст
0,2,887,851,394,342,326,290,0,0,210,...,0,0,0,0,0,0,0,13,1,4
1,2,5324,5138,146,4478,1923,2011,0,0,1110,...,0,0,0,0,4,0,0,13,1,4
2,2,5376,5033,4,4557,1865,1949,0,0,1117,...,0,0,0,0,2,0,0,13,1,4
3,2,5573,5301,1,4775,2007,2008,0,0,1248,...,0,0,0,0,5,0,0,13,1,4
4,2,6138,5983,2,5500,2241,1991,0,7,1440,...,0,0,0,0,1,0,0,13,1,4


##Создание обучающей и тестовой выборок

In [7]:
df.loc[df['рейтинг'] > 0, 'рейтинг'] = 1

In [8]:
scaler = MinMaxScaler()
x = scaler.fit_transform(df.drop('рейтинг', axis = 1).values)
y = df['рейтинг']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle = True, random_state=4)
print('Обучающая выборка:')
print('x: ', x_train.shape)
print('y: ', y_train.shape)
print('Тестовая выборка:')
print('x: ', x_test.shape)
print('y: ', y_test.shape)

Обучающая выборка:
x:  (9442, 50)
y:  (9442,)
Тестовая выборка:
x:  (2361, 50)
y:  (2361,)


#Обучение нейронной сети

In [27]:
def evaluate(model, x_train, y_train, n = 10):
    pred = model.predict(x_train)                                 # Предсказание на тренировочной выборке
    pred = np.round(pred).astype(int)                            # Округление предсказаний до ближайшего целого числа
    pred = np.array(pred)                                       # Преобразуем pred в numpy array

    y_train = y_train.reset_index(drop=True)                     # Сбросить индексы для y_train

    print('Точность:', accuracy_score(y_train, pred), '\n')

    # Печатаем первые n примеров результата
    for i in range(min(n, len(y_train))):
        print('Реальное значение:', y_train[i], ' Предсказанное значение:', pred[i])

In [14]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.6/924.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 40.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: te

In [1]:
import autokeras as ak

Using TensorFlow backend


##Тюнер bayesian

In [11]:
model = ak.StructuredDataClassifier(
    column_names=None,
    column_types=None,
    num_classes=None,
    multi_label=False,
    loss=None,
    metrics=None,
    project_name="usedesk_predict",
    max_trials=10,
    directory='/content/drive/MyDrive/StudyFiles/Usedesk',
    objective="val_accuracy",
    tuner='bayesian',
    overwrite=False,
    seed=None,
    max_model_size=None,
)

model.fit(x_train, y_train, epochs = 150, validation_split = 0.15, verbose=1)

Trial 10 Complete [00h 05m 56s]
val_accuracy: 0.8933236598968506

Best val_accuracy So Far: 0.8933236598968506
Total elapsed time: 01h 01m 44s
Epoch 1/150
296/296 [==============================] - 5s 9ms/step - loss: 0.4370 - accuracy: 0.8380
Epoch 2/150
296/296 [==============================] - 3s 9ms/step - loss: 0.3731 - accuracy: 0.8632
Epoch 3/150
296/296 [==============================] - 5s 17ms/step - loss: 0.3599 - accuracy: 0.8637
Epoch 4/150
296/296 [==============================] - 4s 14ms/step - loss: 0.3520 - accuracy: 0.8655
Epoch 5/150
296/296 [==============================] - 3s 9ms/step - loss: 0.3453 - accuracy: 0.8657
Epoch 6/150
296/296 [==============================] - 3s 10ms/step - loss: 0.3417 - accuracy: 0.8664
Epoch 7/150
296/296 [==============================] - 3s 10ms/step - loss: 0.3392 - accuracy: 0.8678
Epoch 8/150
296/296 [==============================] - 5s 16ms/step - loss: 0.3347 - accuracy: 0.8681
Epoch 9/150
296/296 [=======================

##Тюнер hyperband

In [13]:
model1 = ak.StructuredDataClassifier(
    column_names=None,
    column_types=None,
    num_classes=None,
    multi_label=False,
    loss=None,
    metrics=None,
    project_name="usedesk_predict_1",
    max_trials=10,
    directory='/content/drive/MyDrive/StudyFiles/Usedesk',
    objective="val_accuracy",
    tuner='hyperband',
    overwrite=False,
    seed=None,
    max_model_size=None,
)

model1.fit(x_train, y_train, epochs = 150, validation_split = 0.15, verbose=1)

Trial 10 Complete [00h 00m 39s]
val_accuracy: 0.8708272576332092

Best val_accuracy So Far: 0.876632809638977
Total elapsed time: 00h 06m 58s
Epoch 1/150
296/296 [==============================] - 5s 13ms/step - loss: 0.3970 - accuracy: 0.8590
Epoch 2/150
296/296 [==============================] - 6s 19ms/step - loss: 0.3622 - accuracy: 0.8643
Epoch 3/150
296/296 [==============================] - 3s 10ms/step - loss: 0.3495 - accuracy: 0.8649
Epoch 4/150
296/296 [==============================] - 5s 17ms/step - loss: 0.3367 - accuracy: 0.8681
Epoch 5/150
296/296 [==============================] - 3s 10ms/step - loss: 0.3326 - accuracy: 0.8686
Epoch 6/150
296/296 [==============================] - 4s 12ms/step - loss: 0.3271 - accuracy: 0.8699
Epoch 7/150
296/296 [==============================] - 5s 18ms/step - loss: 0.3198 - accuracy: 0.8711
Epoch 8/150
296/296 [==============================] - 3s 10ms/step - loss: 0.3145 - accuracy: 0.8720
Epoch 9/150
296/296 [=====================

##Тюнер greedy

In [14]:
model2 = ak.StructuredDataClassifier(
    column_names=None,
    column_types=None,
    num_classes=None,
    multi_label=False,
    loss=None,
    metrics=None,
    project_name="usedesk_predict_2",
    max_trials=10,
    directory='/content/drive/MyDrive/StudyFiles/Usedesk',
    objective="val_accuracy",
    tuner='greedy',
    overwrite=False,
    seed=None,
    max_model_size=None,
)

model2.fit(x_train, y_train, epochs = 150, validation_split = 0.15, verbose=1)

Trial 10 Complete [00h 04m 49s]
val_accuracy: 0.9296081066131592

Best val_accuracy So Far: 0.9296081066131592
Total elapsed time: 00h 57m 24s
Epoch 1/150
296/296 [==============================] - 8s 18ms/step - loss: 0.5251 - accuracy: 0.8331
Epoch 2/150
296/296 [==============================] - 3s 10ms/step - loss: 0.4066 - accuracy: 0.8601
Epoch 3/150
296/296 [==============================] - 3s 10ms/step - loss: 0.3791 - accuracy: 0.8633
Epoch 4/150
296/296 [==============================] - 4s 13ms/step - loss: 0.3618 - accuracy: 0.8643
Epoch 5/150
296/296 [==============================] - 6s 19ms/step - loss: 0.3474 - accuracy: 0.8663
Epoch 6/150
296/296 [==============================] - 3s 11ms/step - loss: 0.3379 - accuracy: 0.8650
Epoch 7/150
296/296 [==============================] - 4s 15ms/step - loss: 0.3233 - accuracy: 0.8689
Epoch 8/150
296/296 [==============================] - 6s 20ms/step - loss: 0.3101 - accuracy: 0.8721
Epoch 9/150
296/296 [====================

По итогам обучение лучшие результаты на валидационной выборке показала модель с тюнером greedy. Ее и назовем лучшей

In [15]:
best_model = model2.export_model()
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 multi_category_encoding (M  (None, 50)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 50)                101       
 on)                                                             
                                                                 
 dense (Dense)               (None, 256)               13056     
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [17]:
from tensorflow.keras.models import load_model

In [22]:
import tensorflow as tf

In [23]:
try:
    best_model.save("model_autokeras", save_format="tf")
except Exception:
    best_model.save("model_autokeras.h5")


loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

74/74 [==============================] - 2s 11ms/step
[[1.        ]
 [0.35042295]
 [1.        ]
 ...
 [1.        ]
 [0.9999976 ]
 [0.95472175]]


In [28]:
evaluate(loaded_model, x_test, y_test)

74/74 [==============================] - 1s 7ms/step
Точность: 0.9335027530707327 

Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 0  Предсказанное значение: [0]
Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 1  Предсказанное значение: [0]
Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 1  Предсказанное значение: [1]
Реальное значение: 1  Предсказанное значение: [1]


Итог: на тестовой выборке была получена точность 93%